# Linear Regression
This is a more extended example than `ex02.ipynb` to look at the different ways we can do the same thing in TensorFlow. In particular, this shows one of the major themes in TensorFlow - the trade-off between detailed analytical work done by hand and the abstract pre-fabricated solutions available off-the-shelf.

In [6]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
sess = tf.Session()

## Data

In [2]:
N = 100
P = 1
slope = 5
intercept = 2
X_train = np.random.normal(size=[N, P])
Y_train = X_train * slope + intercept

## Model
Let's define our prediction as:
$$\widehat{y}=x\beta+b\left(\begin{array}{c}1\\1\\\vdots\\1\end{array}\right)=x\beta+b\mathbb{1}_N$$

In [3]:
# Define our data for training / evaluation.
# A tf.placeholder is a recipe for an input. Here, we say that each has the shape [None, 1]
# which means that we are going to provide an arbitrary number of inputs that each are one-dimensional.
x = tf.placeholder(dtype=tf.float32, shape=[None, 1], name="x")
Y = tf.placeholder(dtype=tf.float32, shape=[None, 1], name="y")

# Define the model. This includes our variables (beta and b) and how they interact with our inputs.
# We'll initialize them to 0 (arbitrarily).
beta = tf.Variable(tf.zeros((1, 1)))
b = tf.Variable(tf.zeros((1, 1)))
Y_pred = tf.matmul(x, beta) + b

## Loss and gradient
The loss is the sum of the squared errors:
\begin{align}
\mathcal{L} &= (Y - \hat{y})^\top (Y-\hat{y})
\\
&= (Y-(X\beta+b\mathbb{1}_N))^\top(Y-(X\beta+b\mathbb{1}_N))
\\
&=Y^\top Y - 2 Y^\top (X\beta+b\mathbb{1}_N)+(X\beta+b\mathbb{1}_N)^\top(X\beta+b\mathbb{1}_N)
\end{align}

The gradient of the loss with respect to our variables ($\beta$ and $b$) is:
$$\frac{\partial \mathcal{L}}{\partial \beta} = - 2 X^\top Y + 2 X^\top X \beta + 2 N b \bar{x}$$
$$\frac{\partial \mathcal{L}}{\partial b} = - 2 N \bar{Y} + 2 N \bar{x}^\top\beta + 2 N b$$

Where $\bar{Y}=\frac{1}{N}\sum_i Y_i$ and $\bar{x}_j=\frac{1}{N}\sum_{i=1}^N X_{i,j}$

(as all good write-ups, the derivation of the above is left to the reader)

[Note: We could use the mean squared error instead and pick up a multiplicative factor of $\frac{1}{N}$ everywhere (loss function and each component of the gradient), but the minimum will occur at the same place, so it's somewhat moot as long as our batch size stays constant]

# TensorFlow as calculator
If we know an answer analytically, we can still use TensorFlow to compute it. Because we're encoding the bias term separately (trying to be more like the CS literature of having separate weights and biases when we get to neural networks, but diverging from the standard column-of-ones treatment we use in statistics), we get a slight update for the standard solution to linear regression (that we can derive by setting the gradient [above] to zero):

\begin{align}
\left(X^\top X - N \bar{x}\bar{x}^\top\right)\hat{\beta} &= \left(X^\top Y - N\bar{x}\bar{Y}\right)
\\
\hat{b} &= \bar{y}-\bar{x}^\top \hat{\beta}
\end{align}

In [7]:
xt = tf.transpose(x)
batch_size = tf.cast(tf.shape(x)[0], tf.float32)
x_mean = tf.reshape(tf.reduce_mean(xt, 1), (-1, 1))
Y_mean = tf.reduce_mean(Y)

# Set up the linear equation to be solved, like you might do in R.
lhs = tf.matmul(xt, x) - batch_size * tf.matmul(x_mean, tf.transpose(x_mean))
rhs = tf.matmul(xt, Y) - batch_size * x_mean * Y_mean
# Solved via LU decomposition internally.
beta_hat = tf.matrix_solve(lhs, rhs)
b_hat = Y_mean - tf.matmul(tf.transpose(x_mean), beta_hat)

### Result
And we get a result within $\epsilon$ of the values used to generate the data set:

In [8]:
sess.run([beta_hat, b_hat], {x:X_train, Y:Y_train})

[array([[ 4.99999905]], dtype=float32), array([[ 2.00000024]], dtype=float32)]

## (Stochastic) Gradient Descent
Matrices can be messy, and if our data set is large (especially if $P$, the number of dimensions in the input, is high), we might only be able to do small batches of work at a time. In particular, we resort to gradient descent (or its minibatch cousin stochastic GD) to solve the problem.

One way to do that is to analytically compute the gradients and do a small update each pass.

In [9]:
xt = tf.transpose(x)
x_mean = tf.reshape(tf.reduce_mean(xt, 1), (-1, 1))
Y_mean = tf.reduce_mean(Y)
batch_size = tf.cast(tf.shape(x)[0], tf.float32)

# Gradients
dloss_dbeta = - 2 * tf.matmul(xt, Y) + 2 * tf.matmul(tf.matmul(xt, x), beta) + 2 * batch_size * b * x_mean
dloss_db = - 2 * batch_size * Y_mean + 2 * batch_size * tf.matmul(tf.transpose(x_mean), beta) + 2 * b * batch_size

# Updates
learning_rate = 0.002
new_beta = beta - learning_rate * dloss_dbeta
new_b = b - learning_rate * dloss_db

# Group the updates into a single TensorFlow operation. The tf.assign operation will
# Take the second value and store it in the first Variable.
update_rule = tf.group(tf.assign(beta, new_beta), tf.assign(b, new_b))

In [10]:
sess.run(tf.global_variables_initializer())

### Results
Here, we can run on the full data set (although we could just take slices instead), and show how the parameters quickly converge.

In [11]:
for _ in xrange(20):
    sess.run(update_rule, feed_dict={x: X_train, Y: Y_train})
    beta_np, b_np = sess.run([beta, b], feed_dict={x: X_train, Y: Y_train})
    print(beta_np[0, 0], b_np[0, 0])

(2.0793245, 1.1530027)
(3.2712488, 1.6138525)
(3.9686174, 1.8903623)
(4.3761468, 2.0070336)
(4.6174707, 2.0482645)
(4.7623405, 2.0457377)
(4.8512335, 2.0442216)
(4.9057765, 2.0370359)
(4.9396844, 2.0288739)
(4.9610252, 2.020076)
(4.9747148, 2.0147974)
(4.9834709, 2.0106635)
(4.9891248, 2.0071659)
(4.9928336, 2.0050673)
(4.9952531, 2.0035465)
(4.9968424, 2.0024631)
(4.9978919, 2.0017009)
(4.9985886, 2.0011694)
(4.9990525, 2.0007687)
(4.9993649, 2.0005281)


# There must be a better way
This may seem like a lot of math for a machine learning system that's supposed to "just work". If you thought that, you'd be right.

There's a lot we can do to make this whole process easier.

## Automatic differentiation
One of the ways we could do GD/SGD above a lot easier is if we didn't have to compute the derivative by hand. Well, we're in luck: TensorFlow has the `tf.gradients` function which will do symbolic differentiation with respect to variables in the graph.

In [12]:
loss = tf.reduce_sum((Y - Y_pred) ** 2)
dloss_dbeta, dloss_db = tf.gradients(loss, [beta, b])
# dloss_db = tf.gradients(loss, b)[0]

# Updates
learning_rate = 0.002
new_beta = beta - learning_rate * dloss_dbeta
new_b = b - learning_rate * dloss_db

# Group the updates into a single TensorFlow operation. The tf.assign operation will
# Take the second value and store it in the first Variable.
update_rule = tf.group(tf.assign(beta, new_beta), tf.assign(b, new_b))

### Results
And here we are, getting the same results for the same learning rate, without having to compute the gradient by hand.

In [13]:
sess.run(tf.global_variables_initializer())

In [14]:
for _ in xrange(20):
    sess.run(update_rule, feed_dict={x: X_train, Y: Y_train})
    beta_np, b_np = sess.run([beta, b], feed_dict={x: X_train, Y: Y_train})
    print(beta_np[0, 0], b_np[0, 0])

(2.0793245, 1.1530027)
(3.2712493, 1.6980028)
(3.962677, 1.9408524)
(4.3689451, 2.0377469)
(4.6108928, 2.0672009)
(4.7569761, 2.0677917)
(4.8463922, 2.0578327)
(4.9018512, 2.0455444)
(4.9366808, 2.034256)
(4.9588065, 2.0250239)
(4.9730077, 2.0179226)
(4.9822054, 2.0126593)
(4.9882092, 2.0088518)
(4.9921546, 2.0061436)
(4.9947619, 2.00424)
(4.9964929, 2.0029137)
(4.9976468, 2.0019958)
(4.9984179, 2.0013635)
(4.9989347, 2.0009298)
(4.9992819, 2.000633)


## Removing the explicit update
The next annoying thing in our code above is that we've had to implement gradient descent ourselves. It would be nice if we didn't have to do that ourselves.

TensorFlow has several built-in optimizers:
- Gradient descent
- Adam
- Adagrad
- RMSProp
- etc.

We can use them to compute gradients in a form that's pairs of gradients and variables. That can then be applied back directly. This also gives us the opportunity to update any of the gradients, such as clipping them if they are too large.

In [15]:
loss = tf.reduce_sum((Y - Y_pred) ** 2)
opt = tf.train.GradientDescentOptimizer(learning_rate=0.002)
grads_and_vars = opt.compute_gradients(loss, [beta, b])

# Could introduce clipping here.

# This now does all the assignments at once.
train_op = opt.apply_gradients(grads_and_vars)

### Results

In [16]:
sess.run(tf.global_variables_initializer())

In [17]:
for _ in xrange(20):
    sess.run(train_op, feed_dict={x: X_train, Y: Y_train})
    beta_np, b_np = sess.run([beta, b], feed_dict={x: X_train, Y: Y_train})
    print(beta_np[0, 0], b_np[0, 0])

(2.0793245, 1.1530027)
(3.2712493, 1.6980028)
(3.962677, 1.9408524)
(4.3689451, 2.0377469)
(4.6108928, 2.0672009)
(4.7569761, 2.0677917)
(4.8463922, 2.0578327)
(4.9018512, 2.0455444)
(4.9366808, 2.034256)
(4.9588065, 2.0250239)
(4.9730077, 2.0179226)
(4.9822054, 2.0126593)
(4.9882092, 2.0088518)
(4.9921546, 2.0061436)
(4.9947619, 2.00424)
(4.9964929, 2.0029137)
(4.9976468, 2.0019958)
(4.9984179, 2.0013635)
(4.9989347, 2.0009298)
(4.9992819, 2.000633)


## Simple minimization
If we don't need clipping, there's a simpler method we can use that handles all the work for us: `minimize`.

In [18]:
loss = tf.reduce_sum((Y - Y_pred) ** 2)
# This automatically does symbolic differentiation for each
# Variable in the graph, rather than having to specify
# them all individually.
train_op = (
    tf.train.GradientDescentOptimizer(learning_rate=0.002)
        .minimize(loss)
)

### Results

In [19]:
sess.run(tf.global_variables_initializer())

In [20]:
for _ in xrange(20):
    sess.run(train_op, feed_dict={x: X_train, Y: Y_train})
    beta_np, b_np = sess.run([beta, b], feed_dict={x: X_train, Y: Y_train})
    print(beta_np[0, 0], b_np[0, 0])

(2.0793245, 1.1530027)
(3.2712493, 1.6980028)
(3.962677, 1.9408524)
(4.3689451, 2.0377469)
(4.6108928, 2.0672009)
(4.7569761, 2.0677917)
(4.8463922, 2.0578327)
(4.9018512, 2.0455444)
(4.9366808, 2.034256)
(4.9588065, 2.0250239)
(4.9730077, 2.0179226)
(4.9822054, 2.0126593)
(4.9882092, 2.0088518)
(4.9921546, 2.0061436)
(4.9947619, 2.00424)
(4.9964929, 2.0029137)
(4.9976468, 2.0019958)
(4.9984179, 2.0013635)
(4.9989347, 2.0009298)
(4.9992819, 2.000633)


## Simpler loss
At this point, the only thing in the graph that's not part of the model or something "magical" like the Optimizer is the loss function. Wouldn't it be great if we didn't have to compute that by hand?

In [21]:
loss = tf.losses.mean_squared_error(labels=Y, predictions=Y_pred)
train_op = (
    tf.train.GradientDescentOptimizer(learning_rate=0.2)
        .minimize(loss)
)

### Results

In [22]:
sess.run(tf.global_variables_initializer())

In [23]:
for _ in xrange(20):
    sess.run(train_op, feed_dict={x: X_train, Y: Y_train})
    beta_np, b_np = sess.run([beta, b], feed_dict={x: X_train, Y: Y_train})
    print(beta_np[0, 0], b_np[0, 0])

(2.0793254, 1.1530026)
(3.2712498, 1.6980028)
(3.9626775, 1.9408524)
(4.3689451, 2.0377469)
(4.6108928, 2.0672009)
(4.7569761, 2.0677917)
(4.8463922, 2.0578327)
(4.9018512, 2.0455444)
(4.9366808, 2.034256)
(4.9588065, 2.0250239)
(4.9730077, 2.0179226)
(4.9822054, 2.0126593)
(4.9882092, 2.0088518)
(4.9921546, 2.0061436)
(4.9947619, 2.00424)
(4.9964929, 2.002914)
(4.9976468, 2.001996)
(4.9984179, 2.0013638)
(4.9989347, 2.0009298)
(4.9992819, 2.000633)


## Customization
Now that we have all the quick-and-easy built-in functions in play, we can start using some more advanced techniques:

In [24]:
loss = tf.losses.huber_loss(labels=Y, predictions=Y_pred)
train_op = (
    tf.train.RMSPropOptimizer(learning_rate=0.5)
        .minimize(loss)
)
sess.run(tf.global_variables_initializer())
for _ in xrange(20):
    sess.run(train_op, feed_dict={x: X_train, Y: Y_train})
    beta_np, b_np = sess.run([beta, b], feed_dict={x: X_train, Y: Y_train})
    print(beta_np[0, 0], b_np[0, 0])

(0.39096761, 0.20299864)
(0.79054809, 0.41175276)
(1.1983778, 0.62564063)
(1.614144, 0.84344792)
(2.0375748, 1.0631342)
(2.4682589, 1.2814939)
(2.905431, 1.4897974)
(3.3476224, 1.6850142)
(3.7917595, 1.8693861)
(4.2271633, 2.0307839)
(4.607008, 2.1110363)
(4.8318753, 2.0752444)
(4.9300275, 2.0337627)
(4.9727731, 2.0132186)
(4.990294, 2.0047102)
(4.9968781, 2.0015152)
(4.9991102, 2.0004318)
(4.9997807, 2.0001063)
(4.9999547, 2.0000219)
(4.9999928, 2.0000036)
